In [15]:
from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen
import pandas as pd
import json
import time
import re

# PARTIE 1 : COLLECTE DE DONNEES SUR SPRINGER

In [16]:
# lien de la première page de recherche 
first_url = 'https://rd.springer.com/search?query=multi-label*+AND+OR+AND+multilabel*+AND+AND+AND+text*+AND+OR+AND+document+AND+OR+AND+%E2%80%9Cnatural+AND+language+AND+process*%E2%80%9D+AND+AND+AND+class*+AND+OR+AND+categorization*+AND+OR+AND+tag*&date-facet-mode=between&showAll=true#'

In [26]:
# Collecte le code source d'une page web d'un url donné en argument 
def collecte_source_code(url : str) -> str :
    try : 
        req = Request(url,headers={'User-Agent': 'Chrome/71.0.3578.98'})
        webpage = urlopen(req).read()
        source_code = BeautifulSoup(webpage,'html.parser') 
        return source_code
    except Exception as e :
        print('erreur de chargement de la page... opération recommencée') 
        collecte_source_code(url)

# Cette fonction compte le nombre de page de résultat de recherche pour la clé de recherche 
def collect_link_article_and_conference(first_url : str):
    list_url = []
    list_url.append(first_url + '&facet-content-type=%22Article%22')
    list_url.append(first_url + '&facet-content-type="ConferencePaper')
    return list_url 

# Cette fonction compte le nombre de page de résultat de recherche pour la clé de recherche 
def page_counter(url : str) -> list :
    soup = collecte_source_code(url)
    try :
        number_page  = int(soup.find('span', { "class" : "number-of-pages"}).getText())
    except : 
        number_page = int(1)
    return number_page

# Cette fonctionne transforme la clé de recherche en liste de liens de chaque page de résultat
def collect_all_url_research(url : str) -> list :
    url_list=[url]
    number_page = page_counter(url)
    print(number_page)
    if number_page > 1 :
        second_part_url = (url).replace('https://rd.springer.com/search','')
        for i in range(2, number_page, 1):
            i = str(i)
            url_list.append('https://rd.springer.com/search/page/' + i + second_part_url)
    return url_list

# collecte une liste de titre, date, type de publication, mots clé pour chaque article 
# et crée une dataframe avec ces données
def collect_data(article_list : list, result_type : str) :
    i = 1
    title_list = []
    date_list = []
    keywords_list = []
    publication_type_list = []
    if result_type == 'article':
        for article in article_list : 
            print("chargement des données de l'article", i)
            soup = collecte_source_code(article)
            try :
                title = soup.find('meta', {'name':'dc.title'}).get('content') # title
                title_list.append(title)
            except Exception as e : 
                title=0
                title_list.append(title)
            try : 
                publication_date = soup.find('meta', {'name':'dc.date'}).get('content') # date
                date_list.append(publication_date)
            except Exception as e : 
                publication_date=0
                date_list.append(publication_date)
            try : 
                script = soup.find('script', attrs={'type':'text/javascript'}).text
                script = script.replace('window.dataLayer = ','')
                script = script.replace(';','')
                json1_data = json.loads(script)[0] # transformation du script json en dictionnaire 
                keywords = json1_data['Keywords'] # key words
                keywords_list.append(keywords)
                print(keywords)
            except Exception as e : 
                keywords=0
                keywords_list.append(keywords)
            try :     
                publication_type = soup.find('meta', {'name':'dc.type'}).get('content') # publication type
                publication_type_list.append(publication_type)
            except Exception as e : 
                publication_type = 0 
                publication_type_list.append(publication_type)     
            i=i+1
    else : 
        for article in article_list : 
            print(article)
            print("chargement des données de l'article", i)
            soup = collecte_source_code(article)
            try :
                title = soup.find('meta', {'name':'citation_title'}).get('content') # title
                title_list.append(title)
            except Exception as e : 
                title=0
                title_list.append(title)
            try : 
                publication_date = soup.find('meta', {'name':'citation_publication_date'}).get('content') # date
                date_list.append(publication_date)
            except Exception as e : 
                publication_date=0
                date_list.append(publication_date)
            try : 
                print('keywords',keywords)
                keywords = soup.findAll("span",{'class':"Keyword"})
                keywords = str(keywords)
                keywords = keywords.replace('<span class="Keyword">','')
                keywords = keywords.replace('</span>','')
                keywords = keywords.replace('[','')
                keywords = keywords.replace(']','')
                print(keywords)
                keywords_list.append(keywords)
            except Exception as e : 
                keywords=0
                keywords_list.append(keywords)
            try :     
                publication_type = soup.find('meta', {'name':'dc.type'}).get('content') # publication type
                publication_type_list.append(publication_type)
            except Exception as e : 
                publication_type = 0 
                publication_type_list.append(publication_type)     
            i=i+1
    create_dataframe(title_list, date_list, keywords_list, publication_type_list, result_type)
    

# Création d'une dataframe à partir des listes de titre, date, type de publication, mots clé pour chaque article
def create_dataframe(title_list : list, date_list : list, keywords_list : list, publication_type_list : list, result_type : str) :
       # crée un dictionnaire à partir des listes
        dict = {'Title': title_list, 'Date': date_list , 'Author Keywords': keywords_list, 'Publication type' : publication_type_list}
        # utilisation du dictionnaraire pour créer une dataframe
        df = pd.DataFrame(dict)
        if result_type == 'article':
            df.to_csv('data/dataSPRINGER_article.csv')
        else : 
            df.to_csv('data/dataSPRINGER_conference.csv')
        return df



# collecte les liens des pages de recherche et des articles et en extrait les données 
def scrapping_springer(url : list, result_type : str) :
    #url_list = transform_key_into_url(KEY)
    i = 0
    url_list = url
    article_list = []
    url_list_article = []
    for url in url_list :
        soup = collecte_source_code(url)
        alllink = soup.find_all(class_= 'title') # liens des articles
        i+=1
        print('collecte des liens des articles sur la page', i)
        for link in alllink :
            article_list.append('https://rd.springer.com/' + link['href']) # tout les liens des articles 
    print("le nombre total d'article est de", len(article_list))
    collect_data(article_list, result_type)

In [28]:
# lance le scraping sur springer
first_page_url = collect_link_article_and_conference(first_url)
#scrapping_springer(first_page_url[0])
url_research = collect_all_url_research(first_page_url[0])
df_article = scrapping_springer(url_research,'article')
url_research = collect_all_url_research(first_page_url[1])
df_conference = scrapping_springer(url_research, 'conference')

2
collecte des liens des articles sur la page 1
le nombre total d'article est de 20
chargement des données de l'article 1
chargement des données de l'article 2
chargement des données de l'article 3
Topic models, LDA, Multi-label classification, Document modeling, Text classification, Graphical models, Probabilistic generative models, Dependency-LDA
chargement des données de l'article 4

chargement des données de l'article 5
chargement des données de l'article 6
chargement des données de l'article 7
Multilabel classification, Instance-based learning, Nearest neighbor classification, Logistic regression, Bayesian inference
chargement des données de l'article 8
chargement des données de l'article 9
chargement des données de l'article 10
chargement des données de l'article 11
chargement des données de l'article 12
Text categorization, Deep belief network, Stochastic gradient descent, CAViaR, Vector space model
chargement des données de l'article 13
chargement des données de l'article 14
LS

# PARTIE 2 : GENERATION D'UN CSV COMPTANT LES OCCURENCES DE MOTS-CLE 

In [ ]:
# télécharger le csv
df = pd.read_csv('dataSPRINGER.csv')
# crée une dataframe avec les "Author Keywords"
df_words = df[['Author Keywords']]

In [ ]:
# sépare les valeurs séparées par "," en colonnes 
df_words = df_words['Author Keywords'].str.split(",", expand = True) 

In [ ]:
# regroupe toutes les colonnes en une seule colonne
df_words = df_words.stack().reset_index()

In [ ]:
# supprime les colonnes inutilisées 
df_words.drop(columns=['level_0'], inplace=True)
df_words.drop(columns=['level_1'], inplace=True)

In [ ]:
# supprime les lignes sans valeur
df_words = df_words.dropna()
df_words = df_words[df_words!=0]

In [ ]:
# transformer toutes les lettres en minuscule
df_words = df_words.applymap(lambda s:s.lower() if type(s) == str else s)

In [ ]:
# supprimer les espaces à gauche et à droite des mots 
df_words[0] = (df_words[0]).str.lstrip()
df_words[0] = (df_words[0]).str.rstrip()

In [ ]:
# compte le nombre d'occurence des keywords
df_words = df_words[0].value_counts()

In [ ]:
# télécharge le csv avec le nombre d'occurence par mot
df_words.to_csv('key_words.csv')